In [16]:
df_1 = _dntk.execute_sql(
  'SELECT \n    f.aircraft_type,\n    COUNT(*) AS total_flights\nFROM analytics.aircraft.fct_individual_flights f\nGROUP BY f.aircraft_type\nORDER BY total_flights DESC;',
  'SQL_1D4E6AC3_7A9B_428F_BD7B_9F7E975E2BCF',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_1

,aircraft_type,total_flights
0,Goose,1008
1,Thundercat,553
2,Miniflock,277
3,Bezantium,240
4,Flockinator,192


In [91]:
_dntk.DeepnoteChart(df_1, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"title":"Avions"},"sort":null,"type":"nominal","field":"aircraft_type","scale":{"type":"linear"}},"y":{"axis":{"title":"Nombre de vols","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"total_flights","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"total_flights","scale":{"range":["#2266D3"],"domain":["total_flights"]}},"xOffset":{"datum":"series_0"}},"transform":[]},{"mark":{"dx":0,"dy":-1,"fill":{"expr":"valueLabelsColor"},"type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"axis":{"title":"Avions"},"sort":null,"type":"nominal","field":"aircraft_type","scale":{"type":"linear"}},"y":{"axis":{"title":"Nombre de vols","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"total_flights","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"text":{"type":"quantitative","field":"total_flights","aggregate":"sum"},"color":{"type":"nominal","datum":"total_flights","scale":{"range":["#2266D3"],"domain":["total_flights"]}},"xOffset":{"datum":"series_0"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Quel avion a effectué le plus de vols ? ","config":{"legend":{"orient":"bottom","disable":false}},"params":[{"name":"valueLabelsColor","value":"black"}],"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["total_flights"],"seriesOrder":[0],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

In [97]:
df_2 = _dntk.execute_sql(
  'SELECT\n    airport_code,\n    airport_name,\n    nb_passengers\nFROM analytics.aircraft.fct_airport\nORDER BY nb_passengers DESC;',
  'SQL_1D4E6AC3_7A9B_428F_BD7B_9F7E975E2BCF',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_2

,airport_code,airport_name,nb_passengers
0,AMP,Amazon Mothership,2423400
1,NSA,Nestland Airport,1999700
2,FKT,Flocktopia,1685200


In [298]:
_dntk.DeepnoteChart(df_2, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":false},"encoding":{"x":{"axis":{"grid":false,"title":"Aéroports"},"sort":null,"type":"nominal","field":"airport_name","scale":{"type":"linear"}},"y":{"axis":{"grid":false,"title":"Nombres de passagers","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"nb_passengers","scale":{"type":"linear","domainMin":1500000},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"quantitative","field":"nb_passengers","scale":{"scheme":"deepnote_blues"}},"tooltip":[{"type":"nominal","field":"airport_name"},{"type":"quantitative","field":"nb_passengers","format":{"type":"default","decimals":null}}],"xOffset":{"datum":"series_0"}},"transform":[]},{"mark":{"dx":0,"dy":-1,"fill":{"expr":"valueLabelsColor"},"type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"axis":{"grid":false,"title":"Aéroports"},"sort":null,"type":"nominal","field":"airport_name","scale":{"type":"linear"}},"y":{"axis":{"grid":false,"title":"Nombres de passagers","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"nb_passengers","scale":{"type":"linear","domainMin":1500000},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"text":{"type":"quantitative","field":"nb_passengers"},"color":{"sort":null,"type":"quantitative","field":"nb_passengers","scale":{"scheme":"deepnote_blues"}},"tooltip":[{"type":"nominal","field":"airport_name"},{"type":"quantitative","field":"nb_passengers","format":{"type":"default","decimals":null}}],"xOffset":{"datum":"series_0"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Quel aéroport a été le plus emprunté par les passagers ?","config":{"legend":{"orient":"bottom","disable":false}},"params":[{"name":"valueLabelsColor","value":"black"}],"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Series"],"seriesOrder":[0],"specSchemaVersion":2,"tooltipDefaultMode":false}}""", attach_selection=True, filters='[]')

In [301]:
df_3 = _dntk.execute_sql(
  'WITH airline_year_rpm AS (\n                SELECT\n                    al.airline_name,\n                    EXTRACT(year FROM fs.date) AS flight_year,\n                    SUM(fs.rpm_total) AS total_rpm,\n                    SUM(COALESCE(fs.rpm_domestic, 0)) AS total_rpm_domestic,\n                    SUM(COALESCE(fs.rpm_international, 0)) AS total_rpm_international\n                FROM analytics.aircraft.dim_airline al\n                LEFT JOIN analytics.aircraft.fct_flight_summary fs \n                    ON al.airline_code = fs.airline_code\n                GROUP BY 1, 2\n            ),\n\n            best_rpm AS (\n                SELECT *\n                FROM airline_year_rpm a\n                WHERE total_rpm = (\n                    SELECT MAX(total_rpm)\n                    FROM airline_year_rpm\n                    WHERE airline_name = a.airline_name\n                )\n            )\n\n            SELECT\n                airline_name,\n                flight_year AS best_year,\n                total_rpm,\n                total_rpm_domestic,\n                total_rpm_international\n            FROM best_rpm\n            ORDER BY airline_name;',
  'SQL_1D4E6AC3_7A9B_428F_BD7B_9F7E975E2BCF',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_3

,airline_name,best_year,total_rpm,total_rpm_domestic,total_rpm_international
0,Amazon Airlines,2015,11912594.0,9175044.0,2737550.0
1,Flock Air,2016,17318668.0,13405774.0,3912894.0
2,Goose Airways,2016,49260222.0,34637841.0,14622381.0


In [382]:
_dntk.DeepnoteChart(df_3, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"axis":{"title":"Compagnie aérienne"},"sort":"ascending","type":"nominal","field":"airline_name","scale":{"type":"linear"}},"y":{"axis":{"title":"Années","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"best_year","scale":{"type":"linear","domainMax":2018,"domainMin":2010},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"quantitative","field":"total_rpm","scale":{"scheme":"deepnote_blues"},"aggregate":"sum"},"xOffset":{"datum":"series_0"}},"transform":[]},{"mark":{"dx":0,"dy":-1,"fill":{"expr":"valueLabelsColor"},"type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"axis":{"title":"Compagnie aérienne"},"sort":"ascending","type":"nominal","field":"airline_name","scale":{"type":"linear"}},"y":{"axis":{"title":"Années","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"best_year","scale":{"type":"linear","domainMax":2018,"domainMin":2010},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"text":{"type":"quantitative","field":"best_year"},"color":{"sort":null,"type":"quantitative","field":"total_rpm","scale":{"scheme":"deepnote_blues"},"aggregate":"sum"},"xOffset":{"datum":"series_0"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Quelle a été la meilleure année en RPM pour chaque compagnie aérienne ?","config":{"legend":{"orient":"bottom","disable":false}},"params":[{"name":"valueLabelsColor","value":"black"}],"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["best_year"],"seriesOrder":[0],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

In [385]:
df_4 = _dntk.execute_sql(
  'WITH yearly_asm AS (\n    SELECT\n        al.airline_name,\n        EXTRACT(year FROM fs.date) AS flight_year,\n        AVG(fs.asm_domestic) AS avg_asm_domestic\n    FROM analytics.aircraft.dim_airline al\n    LEFT JOIN analytics.aircraft.fct_flight_summary fs \n        ON al.airline_code = fs.airline_code\n        GROUP BY 1, 2\n),\n\nbest_asm AS (\n    SELECT *\n    FROM yearly_asm a\n    WHERE avg_asm_domestic = (\n        SELECT MAX(avg_asm_domestic)\n        FROM yearly_asm\n        WHERE airline_name = a.airline_name\n    )\n)\n\nSELECT\n    airline_name,\n    flight_year AS best_growth_year,\n    avg_asm_domestic\nFROM best_asm\nORDER BY airline_name;',
  'SQL_1D4E6AC3_7A9B_428F_BD7B_9F7E975E2BCF',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_4

,airline_name,best_growth_year,avg_asm_domestic
0,Amazon Airlines,2002,3.159310e+05
1,Flock Air,2016,4.272554e+05
2,Goose Airways,2016,1.100640e+06


In [562]:
_dntk.DeepnoteChart(df_4, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"circle","tooltip":false},"encoding":{"x":{"axis":{"title":"Capacité de transport"},"sort":null,"type":"quantitative","field":"avg_asm_domestic","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"axis":{"title":"Année","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"best_growth_year","scale":{"type":"linear","zero":false,"domainMax":2018,"domainMin":2000},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"airline_name","scale":{"scheme":"deepnote10"}}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Quelle a été l'année avec le plus de capacité de transport pour chaque compagnie aérienne ?","config":{"legend":{"orient":"bottom","disable":false}},"params":[{"name":"valueLabelsColor","value":"black"}],"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["best_growth_year"],"seriesOrder":[0],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')